## Append Flow

In [0]:
from pyspark.sql.functions import col
# create bronze layer with streaming table

@dlt.table

def bronze_stream_table1():
    # Read streaming data from the specified source table
    df = spark.readStream.table("c1.s1.external_customertable")
    return df


In [0]:
# Bronze layer with customer table2
@dlt.table
def bronze_stream_table2():
    # Read streaming data from the specified source table
    df = spark.readStream.table("c1.s1.t1")
    return df

### Append flow streaming table


In [0]:
import dlt

# create a epty table
dlt.create_streaming_table("Silver_append_flow_table")

# create a append flow
# table1
@dlt.append_flow(
    target="Silver_append_flow_table"
)
def bronze_table1():
    df = spark.readStream.table("bronze_stream_table1")
    return df

# table2
@dlt.append_flow(target="Silver_append_flow_table")
def bronze_table2():
    df=spark.readStream.table("bronze_stream_table2")
    return df  

# Silver layer materialized view
@dlt.table
def silver_mat_view1():
    # Read data from the first bronze table
    df1 = spark.read.table("LIVE.bronze_stream_table1")
    
    # Read data from the second bronze table
    df2 = spark.read.table("LIVE.bronze_stream_table2")
    
    # Perform union of the two DataFrames
    df3 = df1.union(df2)
    
    return df3

In [0]:
# create a gold table

@dlt.table
def gold_append_table():
    df = spark.read.table("LIVE.silver_append_flow_table")
    return df

In [0]:
# Gold layer table materialized table
from pyspark.sql.functions import split, col

@dlt.table
def gold_mat_view1():
    # Read data from the silver materialized view
    df = spark.read.table("LIVE.silver_append_flow_table")
    
    # Add a new column 'domain' by splitting the 'email' column at '@'
    df = df.withColumn("domain", split(col("email"), '@')[1])
    
    return df

In [0]:
# Create a view based on the materialized view

@dlt.view
def silver_view1():
    df = spark.read.table("LIVE.silver_append_flow_table")
    df = df.filter(col("Age") > 20)
    return df

[](https://github.com/KiranSonkamble/deltaLiveTables/blob/main/image.png)

![](image.png)